In [1]:
from modelscope.models import Model
from modelscope.pipelines import pipeline
from modelscope.preprocessors import ConversationalTextToSqlPreprocessor
from modelscope.utils.constant import Tasks

C:\Users\Cherw\Anaconda3\envs\modelscope2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-07 23:50:46,872 - modelscope - INFO - PyTorch version 1.13.1 Found.
2023-05-07 23:50:46,880 - modelscope - INFO - Loading ast index from C:\Users\Cherw\.cache\modelscope\ast_indexer
2023-05-07 23:50:47,238 - modelscope - INFO - Loading done! Current index file version is 1.5.2, with md5 88ae0fd0c912fc3fea3cd73a2c7806b9 and a total number of 860 components indexed


In [2]:
import sqlite3

conn = sqlite3.connect('C:/Users/Cherw/.cache/modelscope/hub/damo/nlp_star_conversational-text-to-sql/db/employee_hire_evaluation/employee_hire_evaluation.sqlite')
cursor = conn.cursor()

#Delete rows from tables
cursor.execute("DELETE FROM employee;",)
cursor.execute("DELETE FROM shop;",)
cursor.execute("DELETE FROM hiring;",)
cursor.execute("DELETE FROM evaluation;",)
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

conn.commit()
tables = cursor.fetchall()
tables

[('employee',), ('shop',), ('hiring',), ('evaluation',)]

In [3]:
import pandas as pd

from IPython.display import display, HTML

for table_name in tables:
    table_name = table_name[0]
    table = pd.read_sql_query("SELECT * from %s" % table_name, conn)
    display (table)

,Employee_ID,Name,Age,City


,Shop_ID,Name,Location,District,Number_products,Manager_name


,Shop_ID,Employee_ID,Start_from,Is_full_time


,Employee_ID,Year_awarded,Bonus


In [6]:
model_id = 'damo/nlp_star_conversational-text-to-sql'
test_case = {
    "database_id": 'employee_hire_evaluation',
    'local_db_path':None,
    "utterance":[
    "I'd like to see Shop names.",
    "Which employee had highest bonus?",
    "Which shop is hiring the highest number of employees? | do you want the name of the shop ? | Yes"]
}

In [5]:
model = Model.from_pretrained(model_id)
preprocessor = ConversationalTextToSqlPreprocessor(model_dir=model.model_dir)
pipeline = pipeline(task=Tasks.table_question_answering,
                    model=model,
                    preprocessor=preprocessor)

2023-05-07 23:52:24,675 - modelscope - INFO - Model revision not specified, use the latest revision: v1.0.0
2023-05-07 23:52:26,397 - modelscope - INFO - initialize model from C:\Users\Cherw\.cache\modelscope\hub\damo\nlp_star_conversational-text-to-sql
2023-05-07 23:52:38 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |

2023-05-07 23:52:38 INFO: Using device: cpu
2023-05-07 23:52:38 INFO: Loading: tokenize
2023-05-07 23:52:38 INFO: Loading: pos
2023-05-07 23:52:38 INFO: Loading: lemma
2023-05-07 23:52:38 INFO: Done loading processors!
2023-05-07 23:52:39 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |

2023-05-07 23:52:39 INFO: Using device: cpu
2023-05-07 23:52:39 INFO: Loading: tokenize
2023-05-07 23:52:39 INFO: Loading: po

In total, process 1 databases .


In [7]:
last_sql = ''
history = []
for item in test_case['utterance']:
    case = {"utterance": item,
            "history": history,
            "last_sql": last_sql,
            "database_id": test_case['database_id'],
            'local_db_path': test_case['local_db_path']}
    results = pipeline(case)
    print(results)
    history.append(item)

punkt has already exist!
{'output': {'text': 'SELECT shop.Name FROM shop'}}
punkt has already exist!
{'output': {'text': 'SELECT evaluation.Employee_ID FROM evaluation ORDER BY evaluation.Bonus DESC LIMIT 1'}}
punkt has already exist!
{'output': {'text': 'SELECT shop.Name FROM hiring JOIN shop GROUP BY hiring.Shop_ID ORDER BY COUNT(*) DESC LIMIT 1'}}
